In [1]:
import ipyleaflet as ipyl
import ipywidgets as ipyw
import json
import numpy as np
import pandas as pd
from shapely.geometry import shape, Point
from ipywidgets import jslink, IntSlider

from traitlets import link

In [2]:
# generate Reindeer positions
nvalues = 10
lons = 7.48 + np.random.rand(nvalues,1)
lons

lats = 60.58 + np.random.rand(nvalues,1)
pd.DataFrame(np.concatenate((lons,lats),axis=1),  columns=["lon", "lat"]).to_csv("reindeers.csv", index=False)

In [3]:
# Read reindeer positions
reindeers = pd.read_csv("reindeers.csv")
reindeers

,lon,lat
0,8.462060,60.896931
1,8.212547,60.661302
2,8.242927,60.926272
3,8.148322,61.132563
4,7.679371,61.256813
5,7.925903,60.979938
6,8.057683,61.150842
7,7.486750,61.444976
8,8.431505,61.082227
9,7.800801,61.356956


In [4]:
lon, lat = 7.48, 60.58
zoom_start = 6
m = ipyl.Map(
#    layers=(basemap_to_tiles(basemaps.OpenTopoMap), ),
    center=(lat, lon)
)
zoom_slider = IntSlider(description='Zoom', min=3, max=17, value=zoom_start)
jslink((zoom_slider, 'value'), (m, 'zoom'))

In [5]:
with open('/home/deeplearning/detect_cwd/nordfjella.json') as f:
    geo_json_data = json.load(f)

In [6]:
layer = ipyl.GeoJSON(data=geo_json_data)
m.add_layer(layer)

In [7]:

# Black if healthy and red if CWD
markers = ()
for lon,lat in zip(reindeers['lon'],reindeers['lat']):
    point = Point(lon,lat)
    outside=True
    for s in geo_json_data['features']:
        polygon = shape(s['geometry'])
        if point.within(polygon):
            print("INSIDE: Raise alarm")
            outside=False
            
    if (outside):
        print("OUTSIDE")  
        markers= markers + (ipyl.Marker(location=(lat,lon)),)
            
marker_cluster = ipyl.MarkerCluster(
        markers=markers
)            
m.add_layer(marker_cluster)

OUTSIDE
INSIDE: Raise alarm
OUTSIDE
OUTSIDE
OUTSIDE
INSIDE: Raise alarm
OUTSIDE
OUTSIDE
OUTSIDE
OUTSIDE


In [8]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [9]:
zoom_slider

IntSlider(value=6, description='Zoom', max=17, min=3)

In [10]:
dc = ipyl.DrawControl()

def handle_draw(self, action, geo_json):
    print(action)
    print(geo_json)

dc.on_draw(handle_draw)
m.add_control(dc)